In [1]:
import cadquery as cq

c = cq.Workplane('front')
c.box(1.2,2,0.4).faces('<X').chamfer(0.1)

In [2]:
import cadquery as cq

cq.Workplane(cq.Plane.XY()).box(4, 2, 0.5).faces(">Z") \
             .workplane().rect(3.5, 1.5, forConstruction=True) \
             .vertices().cskHole(0.125, 0.3, 82.0, depth=None)

In [3]:
#parameter definitions
p_outerWidth = 100.0 #Outer width of box enclosure
p_outerLength = 150.0 #Outer length of box enclosure
p_outerHeight = 50.0 #Outer height of box enclosure

p_thickness =  3.0 #Thickness of the box walls
p_sideRadius =  10.0 #Radius for the curves around the sides of the bo
p_topAndBottomRadius =  2.0 #Radius for the curves on the top and bottom edges of the box

p_screwpostInset = 12.0 #How far in from the edges the screwposts should be place.
p_screwpostID = 4.0 #nner Diameter of the screwpost holes, should be roughly screw diameter not including threads
p_screwpostOD = 10.0 #Outer Diameter of the screwposts.\nDetermines overall thickness of the posts

p_boreDiameter = 8.0 #Diameter of the counterbore hole, if any
p_boreDepth = 1.0 #Depth of the counterbore hole, if
p_countersinkDiameter = 0.0 #Outer diameter of countersink.  Should roughly match the outer diameter of the screw head
p_countersinkAngle = 90.0 #Countersink angle (complete angle between opposite sides, not from center to one side)
p_flipLid = True #Whether to place the lid with the top facing down or not.
p_lipHeight =  1.0 #Height of lip on the underside of the lid.\nSits inside the box body for a snug fit.

#outer shell
oshell = cq.Workplane("XY").rect(p_outerWidth,p_outerLength).extrude(p_outerHeight + p_lipHeight)

#weird geometry happens if we make the fillets in the wrong order
if p_sideRadius > p_topAndBottomRadius:
    oshell.edges("|Z").fillet(p_sideRadius)
    oshell.edges("#Z").fillet(p_topAndBottomRadius)
else:
    oshell.edges("#Z").fillet(p_topAndBottomRadius)
    oshell.edges("|Z").fillet(p_sideRadius)

#inner shell
ishell = oshell.faces("<Z").workplane(p_thickness,True)\
    .rect((p_outerWidth - 2.0* p_thickness),(p_outerLength - 2.0*p_thickness))\
    .extrude((p_outerHeight - 2.0*p_thickness),False) #set combine false to produce just the new boss
ishell.edges("|Z").fillet(p_sideRadius - p_thickness)

#make the box outer box
box = oshell.cut(ishell)

#make the screwposts
POSTWIDTH = (p_outerWidth - 2.0*p_screwpostInset)
POSTLENGTH = (p_outerLength  -2.0*p_screwpostInset)

postCenters = box.faces(">Z").workplane(-p_thickness)\
    .rect(POSTWIDTH,POSTLENGTH,forConstruction=True)\
    .vertices()

for v in postCenters.all():
   v.circle(p_screwpostOD/2.0).circle(p_screwpostID/2.0)\
        .extrude((-1.0)*(p_outerHeight + p_lipHeight -p_thickness ),True)

#split lid into top and bottom parts
(lid,bottom) = box.faces(">Z").workplane(-p_thickness -p_lipHeight ).split(keepTop=True,keepBottom=True).all()  #splits into two solids

#translate the lid, and subtract the bottom from it to produce the lid inset
lowerLid = lid.translate((0,0,-p_lipHeight))
cutlip = lowerLid.cut(bottom).translate((p_outerWidth + p_thickness ,0,p_thickness - p_outerHeight + p_lipHeight))

#compute centers for counterbore/countersink or counterbore
topOfLidCenters = cutlip.faces(">Z").workplane().rect(POSTWIDTH,POSTLENGTH,forConstruction=True).vertices()

#add holes of the desired type
if p_boreDiameter > 0 and p_boreDepth > 0:
    topOfLid = topOfLidCenters.cboreHole(p_screwpostID,p_boreDiameter,p_boreDepth,(2.0)*p_thickness)
elif p_countersinkDiameter > 0 and p_countersinkAngle > 0:
    topOfLid = topOfLidCenters.cskHole(p_screwpostID,p_countersinkDiameter,p_countersinkAngle,(2.0)*p_thickness)
else:
    topOfLid= topOfLidCenters.hole(p_screwpostID,(2.0)*p_thickness)

#flip lid upside down if desired
if p_flipLid:
    topOfLid.rotateAboutCenter((1,0,0),180)

result =topOfLid.combineSolids(bottom)
    
#return the combined result
result

/nix/store/y9dxm7s0xab0ab1z929b9n7labwkg3h6-python3-3.11.6-env/lib/python3.11/site-packages/cadquery/utils.py:39: FutureWarning: combineSolids will be removed in the next release.
  warn(f"{f.__name__} will be removed in the next release.", FutureWarning)


In [4]:
#####
# Inputs
######
lbumps = 6       # number of bumps long
wbumps = 2       # number of bumps wide
thin = True      # True for thin, False for thick

#
# Lego Brick Constants-- these make a lego brick a lego :)
#
pitch = 8.0
clearance = 0.1
bumpDiam = 4.8
bumpHeight = 1.8
if thin:
    height = 3.2
else:
    height = 9.6

t = (pitch - (2 * clearance) - bumpDiam) / 2.0
postDiam = pitch - t  # works out to 6.5
total_length = lbumps*pitch - 2.0*clearance
total_width = wbumps*pitch - 2.0*clearance

# make the base
s = cq.Workplane("XY").box(total_length, total_width, height)

# shell inwards not outwards
s = s.faces("<Z").shell(-1.0 * t)

# make the bumps on the top
s = s.faces(">Z").workplane(). \
    rarray(pitch, pitch, lbumps, wbumps, True).circle(bumpDiam / 2.0) \
    .extrude(bumpHeight)

# add posts on the bottom. posts are different diameter depending on geometry
# solid studs for 1 bump, tubes for multiple, none for 1x1
tmp = s.faces("<Z").workplane(invert=True)

if lbumps > 1 and wbumps > 1:
    tmp = tmp.rarray(pitch, pitch, lbumps - 1, wbumps - 1, center=True). \
        circle(postDiam / 2.0).circle(bumpDiam / 2.0).extrude(height - t)
elif lbumps > 1:
    tmp = tmp.rarray(pitch, pitch, lbumps - 1, 1, center=True). \
        circle(t).extrude(height - t)
elif wbumps > 1:
    tmp = tmp.rarray(pitch, pitch, 1, wbumps - 1, center=True). \
        circle(t).extrude(height - t)
else:
    tmp = s
s

In [5]:
(L,w,t) = (20.0, 6.0, 3.0)
s = cq.Workplane("XY")

#draw half the profile of the bottle and extrude it
p = s.center(-L/2.0, 0).vLine(w/2.0) \
    .threePointArc((L/2.0, w/2.0 + t),(L, w/2.0)).vLine(-w/2.0) \
    .mirrorX().extrude(30.0,True)

#make the neck
p.faces(">Z").workplane().circle(3.0).extrude(2.0,True)

#make a shell
result = p.faces(">Z").shell(0.3)

result

In [6]:
import cadquery as cq

result = cq.Workplane("XY").box(2, 2, 2).\
    faces(">Z").shell(-0.2).\
    faces(">Z").edges("not(<X or >X or <Y or >Y)").\
    chamfer(0.125, 0.02)
    
result

In [7]:
result = cq.Workplane("front").box(3, 2, 0.5)                 #make a basic prism
result = result.faces("<X").workplane(offset=0.75)       #workplane is offset from the object surface
result = result.circle(1.0).extrude(0.5)                 #disc

result